# LEAPS Data Exploration

The LEAPS data is a collection of complete programs in the LEAPS Karel DSL for controlling an agent in the Karel environment.
My goal is to explore how the programs in the dataset perform on the Karel tasks to serve as a baseline dataset. This
will be compared against the dataset being generated using a random policy for the Karel tasks.

## Imports

In [ ]:
import tables as tb

## Read data

In [ ]:
leaps = tb.File('leaps/data/data-001.hdf5', 'r')

In [ ]:
# Explore hdf5 file layout and data
count = 0

for idx, node in enumerate(leaps.walk_nodes()):
    count += 1

print(count)

In [ ]:
data_info = leaps.get_node('/data_info')
print(data_info._f_list_nodes())

In [ ]:
min_program_length = leaps.get_node('/data_info/min_program_length').read()
print(min_program_length)

In [ ]:
max_program_length = leaps.get_node('/data_info/max_program_length').read()
print(max_program_length)

In [ ]:
p1 = leaps.get_node('/no_0_prog_len_19_max_s_h_len_5')
print(p1._f_list_nodes())

In [ ]:
p1_prog = leaps.get_node('/no_0_prog_len_19_max_s_h_len_5')
prog = p1_prog._f_get_child('program').read()
print(prog)

## Produce dataset

In [ ]:
import sys

sys.path.insert(0, 'leaps')

from leaps.prl_gym.exec_env import ExecEnv2
from leaps.pretrain.get_karel_config import get_karel_task_config

In [ ]:
programs = []
tasks = ['cleanHouse', 'harvester', 'fourCorners', 'randomMaze', 'stairClimber', 'topOff']
rewards = { task: [] for task in tasks }

for idx, node in enumerate(leaps.walk_nodes()):
    if 'program' in node:
        prog_node = node
        program_seq = prog_node._f_get_child('program').read()
        for idx, task in enumerate(tasks):
            karel_env = ExecEnv2(get_karel_task_config(task))
            if idx == 0:
                program_str = karel_env.dsl.intseq2str(program_seq)
                programs.append(program_str)
            reward, _ = karel_env.reward(program_seq)
            rewards[task].append(reward)

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'programs': programs,
    'cleanHouse_reward': rewards['cleanHouse'],
    'harvester_reward': rewards['harvester'],
    'fourCorners_reward': rewards['fourCorners'],
    'randomMaze_reward': rewards['randomMaze'],
    'stairClimber_reward': rewards['stairClimber'],
    'topOff_reward': rewards['topOff'],
})
print(df.describe())
df.to_csv('leaps_data.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('leaps_data.csv')
df.describe()

In [ ]:
def prog_len(prog):
    env = ExecEnv2(config=get_karel_task_config('harvester'))
    prog_seq = env.dsl.str2intseq(prog)
    return len(prog_seq)

df['prog_len'] = df['programs'].apply(lambda prog: prog_len(prog))
df.describe()

In [ ]:
def new_harvester(prog):
    env = ExecEnv2(config=get_karel_task_config('harvester'))
    reward, _ = env.reward(prog, is_program_str=True)
    return reward

df['harvester_reward'] = df['programs'].apply(new_harvester)
df.describe()

In [ ]:
df.to_csv('leaps_data.csv')

In [ ]:
import matplotlib.pyplot as plt

num_bins = 100

# Create a figure and arrange subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))

# Plot the distributions in each subplot
axes[0, 0].hist(df.cleanHouse_reward, bins=num_bins, color='blue', alpha=0.7)
axes[0, 0].set_title('cleanHouse')

axes[0, 1].hist(df.harvester_reward, bins=num_bins, color='orange', alpha=0.7)
axes[0, 1].set_title('harvester')

axes[0, 2].hist(df.fourCorners_reward, bins=num_bins, color='green', alpha=0.7)
axes[0, 2].set_title('fourCorners')

axes[1, 0].hist(df.randomMaze_reward, bins=num_bins, color='red', alpha=0.7)
axes[1, 0].set_title('randomMaze')

axes[1, 1].hist(df.stairClimber_reward, bins=num_bins, color='purple', alpha=0.7)
axes[1, 1].set_title('stairClimber')

axes[1, 2].hist(df.topOff_reward, bins=num_bins, color='brown', alpha=0.7)
axes[1, 2].set_title('topOff')

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
anomalous = df[df.filter(like='_reward').apply(lambda col: (col < 0) | (col > 1)).any(axis=1)]
print(anomalous)

anomalous.to_csv('anomalous.csv')

In [ ]:
tasks = ['cleanHouse', 'harvester', 'fourCorners', 'randomMaze', 'stairClimber', 'topOff']
df[df['harvester_reward'] == df['harvester_reward'].min()].programs.values[0]

In [ ]:
max_len = df[df['prog_len'] == df['prog_len'].max()].programs.values[0]
print('\n'.join([f'{idx}, {token}' for idx, token in enumerate(max_len.split())]))

In [ ]:
from collections import Counter
print(Counter(df['fourCorners_reward']))